In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )

In [ ]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

In [ ]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]

print_ww(email)

In [ ]:
get_weather_description = """\
<tool_description>
<tool_name>get_weather</tool_name>
<parameters>
<name>latitude</name>
<name>longitude</name>
</parameters>
</tool_description>
"""

get_lat_long_description = """
<tool_description>
<tool_name>get_lat_long</tool_name>
<parameters>
<name>place</name>  
</parameters>
</tool_description>"""

list_of_tools_specs = [get_weather_description, get_lat_long_description]
tools_string = ''.join(list_of_tools_specs)
print(tools_string)

In [ ]:
from langchain import PromptTemplate

TOOL_TEMPLATE = """\
Your job is to formulate a solution to a given <user-request> based on the instructions and tools below.

Use these Instructions: 
1. In this environment you have access to a set of tools and functions you can use to answer the question.
2. You can call the functions by using the <function_calls> format below.
3. Only invoke one function at a time and wait for the results before invoking another function.
4. The Results of the function will be in xml tag <function_results>. Never make these up. The values will be provided for you.
5. Only use the information in the <function_results> to answer the question.
6. Once you truly know the answer to the question, place the answer in <answer></answer> tags. Make sure to answer in a full sentence which is friendly.
7. Never ask any questions

<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

<user-request>
{user_input}
</user-request>

Human: What is the first step in order to solve this problem?

Assistant:
"""
TOOL_PROMPT = PromptTemplate.from_template(TOOL_TEMPLATE)

In [ ]:
import xmltodict
import json

def invoke_model(prompt):
    client = boto3.client(service_name='bedrock-runtime', region_name=os.environ.get("AWS_REGION"),)
    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 500, "temperature": 0,})
    modelId = "anthropic.claude-v2"
    #modelId = "anthropic.claude-instant-v1"
    response = client.invoke_model(
        body=body, modelId=modelId, accept="application/json", contentType="application/json"
    )
    return json.loads(response.get("body").read()).get("completion")

def single_agent_step(prompt, output):

    # first check if the model has answered the question
    done = False
    if '<answer>' in output:
        answer = output.split('<answer>')[1]
        answer = answer.split('</answer>')[0]
        done = True
        return done, answer
    
    # if the model has not answered the question, go execute a function
    else:

        # parse the output for any 
        function_xml = output.split('<function_calls>')[1]
        function_xml = function_xml.split('</function_calls>')[0]
        function_dict = xmltodict.parse(function_xml)
        func_name = function_dict['invoke']['tool_name']
        parameters = function_dict['invoke']['parameters']

        #print(f"single_agent_step:: func_name={func_name}::params={parameters}::function_dict={function_dict}::")
        # call the function which was parsed
        func_response = call_function(func_name, parameters)

        # create the next human input
        func_response_str = '\n\nHuman: Here is the result from your function call\n\n'
        func_response_str = func_response_str + f'<function_results>\n{func_response}\n</function_results>'
        func_response_str = func_response_str + '\n\nIf you know the answer, say it. If not, what is the next step?\n\nAssistant:'

        # augment the prompt
        prompt = prompt + output + func_response_str
    return done, prompt

In [ ]:
user_input = 'What is the weather in Las Vegas?'
next_step = TOOL_PROMPT.format(tools_string=tools_string, user_input=user_input)

output = invoke_model(next_step).strip()
done, next_step = single_agent_step(next_step, output)
if not done:
    display(Pretty(f'{output}'))
else:
    display(Pretty('Final answer from LLM:\n'+f'{next_step}'))

In [ ]:
output = invoke_model(next_step).strip()
done, next_step = single_agent_step(next_step, output)
if not done:
    display(Pretty(f'{output}'))
else:
    display(Pretty('Final answer from LLM:\n'+f'{next_step}'))

In [ ]:
from langchain.tools import tool
import ast

@tool ("get_lat_long")
def get_lat_long(place: str) -> dict:
    """Returns the latitude and longitude for a given place name as a dict object of python."""
    url = "https://nominatim.openstreetmap.org/search"

    params = {'q': place, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params).json()

    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None

@tool ("get_weather_dict")
def get_weather_dict(co_ord_str: str) -> dict:
    """
    Returns weather data for a given a dictionary having latitude and longitude.
    """
    if '{' in co_ord_str:
        co_ord_str = ast.literal_eval(co_ord_str)
        latitude = co_ord_str['latitude']
        longitude = co_ord_str['longitude']
    else:
        latitude = float(co_ord_str.split(",")[0].strip())
        longitude = float(co_ord_str.split(",")[1].strip())
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    return response.json()

In [ ]:
from langchain import LLMMathChain

math_tool_template = """\
Given a question from the Human with a math problem, provide only a single line mathematical expression that solves the problem in the following format.
Never solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

llm_math_chain = LLMMathChain.from_llm(llm=agent_llm, verbose=True)
llm_math_chain.llm_chain.prompt.template = math_tool_template
result = llm_math_chain('What is nine times 465?')

In [ ]:
from langchain import LLMMathChain

math_tool_template = """\
Given a question from the Human with a math problem, provide only a single line mathematical expression that solves the problem in the following format.
Never solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

llm_math_chain = LLMMathChain.from_llm(llm=agent_llm, verbose=True)
llm_math_chain.llm_chain.prompt.template = math_tool_template
result = llm_math_chain('What is nine times 465?')

In [ ]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [ "get_lat_long", "get_weather_dict", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

react_agent.agent.llm_chain.prompt.template=prompt_template

In [ ]:
result = react_agent("Can you check the weather in Las Vegas for me?")